If additional cations are included, they may be included as (screened) Coulomb interactions with additional LJ/Buckingham potentials.
ref: https://pubs.rsc.org/en/Content/ArticleLanding/2011/CP/c1cp21034c

Solubility of CO2 is approximately 1g/kg H2O at 300 K, so there is approximately 2500 water molecules per molecule of CO2.

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import scipy.constants as cst
from lammpsrun import LAMMPS, Prism

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write
from ase.build import *
from ase.md import *

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
homepath

Populating the interactive namespace from numpy and matplotlib


'D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\'

In [2]:
atoms = molecule('H2O')

box_length = ((125 / (55.5 * cst.Avogadro)) * (1e+27))**(1 / 3.)

atoms.set_cell((box_length, box_length, box_length))
atoms.center()

atoms = atoms.repeat((5, 5, 5))
atoms.set_pbc(True)

view(atoms)

In [37]:
# SPC/E input from http://www.orsi.sems.qmul.ac.uk/downloads.html
Fe_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 55.845"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' Fe'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax iso 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

AttributeError: 'str' object has no attribute 'get_forces'